In [1]:
import gradient

/opt/homebrew/Caskroom/miniconda/base/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gradient.utils.list_models()

2024-03-03 13:48:08.065 |      ERROR       | Failed to list deltas from S3: An error occurred (InvalidAccessKeyId) when calling the ListObjectsV2 operation: The AWS Access Key Id you provided does not exist in our records.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/const/Code/dtrain/gradient/utils.py:220 in list_models                                    │
│                                                                                                  │
│   217 def list_models() -> Dict[int, SimpleNamespace]:                                           │
│   218 │   try:                                                                                   │
│   219 │   │   s3client: boto3.client = client()                                                  │
│ ❱ 220 │   │   response = s3client.list_objects_v2(Bucket=bucket_name, Prefix='hash_')            │
│   221 │   │   deltas_info: Dict[int, Dict[str, typing.Union[str, int]]] = {}                     │
│   222 │   │                                                                                      │
│   223 │   │   for obj in response.get('Contents', []):                                           │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/envs/311/lib/python3.11/site-packages/botocore/client.py:5 │
│ 53 in _api_call                                                                                  │
│                                                                                                  │
│    550 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    551 │   │   │   │   )                                                                         │
│    552 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  553 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    554 │   │                                                                                     │
│    555 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    556                                                                                           │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/envs/311/lib/python3.11/site-packages/botocore/client.py:1 │
│ 009 in _make_api_call                                                                            │
│                                                                                                  │
│   1006 │   │   │   │   "Code"                                                                    │
│   1007 │   │   │   )                                                                             │
│   1008 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1009 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1010 │   │   else:                                                                             │
│   1011 │   │   │   return parsed_response                                                        │
│   1012                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ClientError: An error occurred (InvalidAccessKeyId) when calling the ListObjectsV2 operation: The AWS Access Key Id
you provided does not exist in our records.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 gradient.utils.list_models()                                                                 │
│   2                            

In [4]:
batches = get_random_batches( n = 1, batch_size = 1, sequence_length = 512 )


In [5]:
delta = pull_model( 0 )
print (hash_model(delta))


e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855


In [13]:
print (hash_model(model))

d66efa2d5f09a51ca873d621f030db89f2b17d7647ddc17c5c645ff2600e4087


In [2]:
list()

{0: namespace(uid=0,
           module_name='module_0.pt',
           hash_name='hash_0.txt',
           remote_timestamp='2024-02-22 03:07:00',
           seconds=1708571220,
           remote='2dddfae8d8f0b2cb2f3195c5561e2449d214c5e1fe69195a486142bc5e31833f',
           local='2dddfae8d8f0b2cb2f3195c5561e2449d214c5e1fe69195a486142bc5e31833f',
           stale=False),
 1: namespace(uid=1,
           module_name='module_1.pt',
           hash_name='hash_1.txt',
           remote_timestamp='2024-02-21 07:50:16',
           seconds=1708501816,
           remote='e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
           local='e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
           stale=False),
 10: namespace(uid=10,
           module_name='module_10.pt',
           hash_name='hash_10.txt',
           remote_timestamp='2024-02-22 02:20:31',
           seconds=1708568431,
           remote='d66efa2d5f09a51ca873d621f030db89f2b17d7647ddc17c5c645ff26

In [21]:
import torch

def calculate_delta(model_a: torch.nn.Module, model_b: torch.nn.Module) -> torch.nn.Module:
    """
    Calculates the delta between two models' state_dicts and returns it as a new module.

    Args:
        model_a (torch.nn.Module): The first model.
        model_b (torch.nn.Module): The second model, to be subtracted from the first.

    Returns:
        torch.nn.Module: A new module representing the delta between model_a and model_b.
    """
    delta_dict = {}
    state_dict_a = model_a.state_dict()
    state_dict_b = model_b.state_dict()
    
    for name, param in state_dict_a.items():
        if name in state_dict_b:
            # Subtract the parameters of model_b from model_a and store it in the delta dictionary
            delta_param = param - state_dict_b[name]
            delta_dict[name] = delta_param
    
    import copy
    delta = copy.deepcopy( model_a )
    delta.load_state_dict( delta_dict )
    return delta

calculate_delta( model, model )


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [11]:
model

<All keys matched successfully>

In [7]:

add_delta( model, delta )
print (hash_model(model))

d66efa2d5f09a51ca873d621f030db89f2b17d7647ddc17c5c645ff2600e4087


In [5]:
load_hash( 11 )

'd66efa2d5f09a51ca873d621f030db89f2b17d7647ddc17c5c645ff2600e4087'

In [4]:
download_hash( 11 )

'd66efa2d5f09a51ca873d621f030db89f2b17d7647ddc17c5c645ff2600e4087'

In [22]:
pull_delta( 10 )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3661057/4122668625.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3661057/4122668625.py'                      │
│                                                                                                  │
│ /home/setup/gradient/utils.py:203 in pull_delta                                                  │
│                                                                                                  │
│   200                                                                                            │
│   201 def pull_model( uid: int ) -> torch.nn.Module:                                             │
│   202 │   if load_hash( uid ) != download_hash( uid ):                                           │
│ ❱ 203 │   │   save_model( uid, download_model( uid ) )                                           │
│   204 │   return load_model( uid )                                                               │
│   205                                                                                            │
│   206 def push_model( uid: int, model: torch.nn.Module ):                                        │
│                                                                                                  │
│ /home/setup/gradient/utils.py:151 in save_model                                                  │
│                                                                                                  │
│   148 │   │   return None                                                                        │
│   149                                                                                            │
│   150 def save_model( uid: int, module: torch.nn.Module ):                                       │
│ ❱ 151 │   model_path = model_path(uid)                                                           │
│   152 │   hash_path = hash_path(uid)                                                             │
│   153 │   try:                                                                                   │
│   154 │   │   # Check if we should update the model                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnboundLocalError: local variable 'model_path' referenced before assignment